# Exploratory data analysis on Youtube search and watch history

Veera Määttänen 5.8.2023

The idea behind this project is to gain more knowledge on data analysis and using Python libraries and packages such as numpy, pandas, matplotlib and seaborn. 

- What do I want to obtain from the data? 
- What's the problem or situation that I am trying to solve or understand?
- Other questions?

#### 1. From html to csv files

In [39]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [14]:
with open("watch_history_1.html", encoding='utf8') as fp:
    soup = BeautifulSoup(fp, 'html.parser')

In [67]:
articles = soup.find_all('div', class_='content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1')
article.contents

[<a href="https://www.youtube.com/watch?v=X19vITdE8JY">MILEY CYRUS  " 7 things " make up inspired tutorial</a>,
 '\xa0katsottiin',
 <br/>,
 <a href="https://www.youtube.com/channel/UCUmC122H07lU5BWK7XZimhg">Panacea81 - Lauren Luke</a>,
 <br/>,
 '7.7.2011 klo 22.44.25 EEST']

I had to do some error handling here because the contents of the article were different for hidden and deleted videos. 

- First I tried to implement try except but it didn't work the way I wanted.
- I noticed that the lenght of the contents was 6 for all the videos that are still up and decided to use that as a condition. 
- There were two types of missing datas, deleted videos and hidden/private videos. Deleted videos didn't have any information about the channel or the video, but i wanted to keep them because they did have the date data and i am also interested in that. 
- Some of the hidden/private videos had working links and i was able to watch the videos, but i decided it would make more sense to just label these as hidden or private, and also suitable for this situation.

In [87]:
watch_history = []
for article in articles:
    if len(article.contents) == 6:
        video = article.find('a')
        title = video.string
        channel = title.find_next('a').string
        date = article.contents[-1]
        watch_history.append([title, channel, date])
    elif len(article.contents) == 3: # Deleted videos
        title = article.find_next(string=True)
        channel = float('nan')
        date = article.contents[-1]
        watch_history.append([title, channel, date])
    else: # Hidden or private videos
        title = 'Private/hidden video'
        channel = float('nan')
        date = article.contents[-1]
        watch_history.append([title, channel, date])

In [88]:
df = pd.DataFrame(watch_history, columns=['title','channel','date'])
df.to_csv('watch_history.csv')

#### 2. 

In [89]:
import numpy as np


In [90]:
df.head()

,title,channel,date
0,6. tammikuuta 2022,Jyrki Hakkarainen,1.6.2023 klo 22.10.05 EEST
1,DATA ANALYST PORTFOLIO | 10 PROJECT IDEAS,Data With Mo,1.6.2023 klo 21.46.17 EEST
2,Q&A with a person who does not have an interna...,PA Struggles,1.6.2023 klo 18.49.48 EEST
3,Four Common MISTAKES To AVOID If You Want To L...,Doctor Youn,1.6.2023 klo 18.49.44 EEST
4,How Rainbolt Identifies Countries #geoguessr #...,Profoundly Pointless,1.6.2023 klo 17.23.28 EEST


In [92]:
df.shape

(39445, 3)

In [93]:
df.describe()

,title,channel,date
count,39445,34130,39445
unique,29747,11292,39431
top,Private/hidden video,PewDiePie,19.12.2021 klo 0.34.29 EEST
freq,4174,1596,2


My most watched channel is Pewdiepie and i have watched 1596 of his videos. This doesn't surprise me because I have watched him for a long time and absolutely love his content. 

In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39445 entries, 0 to 39444
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    39445 non-null  object
 1   channel  34130 non-null  object
 2   date     39445 non-null  object
dtypes: object(3)
memory usage: 924.6+ KB


Next I want to split the date into months and years and also times. 